In [191]:
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [157]:
start = time.time()

headers = {
    'accept': '*/*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
'content-length': 1630,
'content-type': 'application/json',
'cookie': 'PHPSESSID=7118cf15ef43e5a19371b5d9f9705b52; csrf=526df025e198b2c438ee645615b85862; fbcity=4; fre=0; rd=1380000; zl=en; fbtrack=4cfc286e16f4fdad8dbb5604da264263; ltv=4; lty=4; locus=%7B%22addressId%22%3A0%2C%22lat%22%3A12.971606%2C%22lng%22%3A77.594376%2C%22cityId%22%3A4%2C%22ltv%22%3A4%2C%22lty%22%3A%22city%22%2C%22fetchFromGoogle%22%3Afalse%2C%22dszId%22%3A3655%2C%22fen%22%3A%22Bengaluru%22%7D; cid=2c4e3ed9-0308-4d16-a237-3a5c99f7e944; G_ENABLED_IDPS=google; expab=1; dpr=1; _gid=GA1.2.339128363.1667474206; _gcl_au=1.1.883744782.1667474206; _fbp=fb.1.1667474210447.1958226114; _ga_2XVFHLPTVP=GS1.1.1667510127.6.1.1667512677.0.0.0; _ga=GA1.1.929623576.1667474206; AWSALBTG=WTYQxvN7H1nBiQ1mh8Da/FETLKSWzLFMCSVLO5VycDlaS1UYiyS7En2MSlhVn5VpvbDXclvR4ixL7dXOHw3SKV9qEKw6gLPVJX4LkI8E7Q5Q0zyyOhNQIy1hztlIKjRMo0N2w8mDvPpyChiksz40+DWkk+dh6KeBCi4sjQQvYTg9; AWSALBTGCORS=WTYQxvN7H1nBiQ1mh8Da/FETLKSWzLFMCSVLO5VycDlaS1UYiyS7En2MSlhVn5VpvbDXclvR4ixL7dXOHw3SKV9qEKw6gLPVJX4LkI8E7Q5Q0zyyOhNQIy1hztlIKjRMo0N2w8mDvPpyChiksz40+DWkk+dh6KeBCi4sjQQvYTg9',
'origin': 'https://www.zomato.com',
'referer': 'https://www.zomato.com/bangalore',
'sec-fetch-dest': 'empty',
'sec-fetch-mode': 'cors',
'sec-fetch-site': 'same-origin',
'sec-gpc': 1,
'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
'x-zomato-csrft': '526df025e198b2c438ee645615b85862'
}

urls = []
rest_name = []
ratings = []
price = []
cuisine = []

driver = webdriver.Chrome(
    ChromeDriverManager().install())  # this might need a change since i am working in pycharm you might need to import os
for namee in ['indiranagar','marathahlli','whitefield','kormangala','hsr','jayanagar','jp nagar','sarjapur road','lavelle road','malleshwaram','btm','mg road','kalyan nagar','electronic city','bannerghatta road','bellandur','new bel road','residency road','church street','brigade road','old airport road','banashankari','basavanagudi','rajajinagar','brookefield','kammanahalli']:
    driver.get(
        f"https://www.zomato.com/bangalore/{namee.replace(' ','-')}-restaurants")  # url of the website of particular cities you want to scrap.
    driver.refresh()

    time.sleep(2)  # Allow 2 seconds for the web page to (open depends on you)
    scroll_pause_time = 3  # You can set your own pause time. dont slow too slow that might not able to load more data
    screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break


    # creating soup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    divs = soup.findAll('div', class_='jumbo-tracker')

    # create all the list here according to data needs
    # zomato is very anti-scrapping website it changes class of the data dynamically (jumbo tracker is the only fixed things i can find)

        #  name of the restaurant is stored in the h4 tags and luckily it was unique in main class
    for parent in divs:  # zomato is very anti-scrapping website it changes class of the data dynamically (jumbo tracker is the only fixed things i can find)

    #  name of the restaurant is stored in the h4 tags and luckily it was unique in main class
        name_tag = parent.find("h4")

        # appending the name to rest_name list
        if name_tag == None:
            continue
        else:
            rest_name.append(name_tag.text)

        #  links of the restaurants are in a tags hence we are using it find and then getting href where links are stored
        link_tag = parent.find("a")

        base = "https://www.zomato.com"  # since we don not get whole link https attached we need to join the link with base
        try:
            if 'href' in link_tag.attrs:
                link = link_tag.get('href')
        except:
            pass
        url = urljoin(base, link)
        urls.append(url)

    for div in divs:
        if div.div.a == None:
            continue
        else:
            if div.div.a.next_sibling == None:
                continue
            else:
                rating_tag = div.a.next_sibling.div.div.div.div.div.div.div.text
                cuisine_tag = div.div.a.next_sibling.div.next_sibling.p.text
                price_tag = div.div.a.next_sibling.div.next_sibling.p.next_sibling.text
        
                ratings.append(rating_tag)
                price.append(price_tag)
                cuisine.append(cuisine_tag)
    
#     for parent in divs:
#         links = parent.select_one('a')
#         if aa == None:
#             continue
#         elif '?' in aa.get('href'):
#             continue
#         else:
#             urls.append(aa.get('href'))
#             print(urls)
            
#         rest_name = i.select_one('h4')
#         if name == None:
#             continue
#         else:
#             print(name.text)
            
        
driver.close()

/tmp/ipykernel_6550/1204787526.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [158]:
len(urls)

9125

In [159]:
len(rest_name)

9125

In [160]:
len(ratings)

9125

In [161]:
len(cuisine)

9125

In [162]:
len(price)

9125

In [163]:
table1 = {'restaurant':rest_name, 'link' : urls, 'cusine' : cuisine, 'price' : price, 'rating' : ratings}

In [165]:
dff = pd.DataFrame(table1)

In [167]:
dff[dff['restaurant'] == 'M.M.A Kabab and Rolls' ]

,restaurant,link,cusine,price,rating
880,Karim's - Original From Delhi 6 Jama Masjid,https://www.zomato.com/bangalore/karims-origin...,"Mughlai, North Indian, Rolls, Kebab",₹150 for one,3.5
885,Restaurant Chef Pillai,https://www.zomato.com/bangalore/restaurant-ch...,"South Indian, Seafood, Biryani, Beverages, Des...",₹150 for one,4.2
904,Copper Chimney,https://www.zomato.com/bangalore/copper-chimne...,"North Indian, Mughlai, Biryani, Beverages",₹400 for one,3.9
1456,Al Daaz,https://www.zomato.com/bangalore/al-daaz-hsr/o...,"North Indian, Mughlai, Continental, Fast Food,...",₹200 for one,4.3
1790,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹100 for one,3.6
...,...,...,...,...,...
9102,Corner House Ice Creams,https://www.zomato.com/bangalore/corner-house-...,"Ice Cream, Desserts, Shake",₹250 for one,4.5
9106,Imperial Restaurant,https://www.zomato.com/bangalore/imperial-rest...,"Chinese, North Indian, South Indian, Seafood, ...",₹250 for one,3.8
9112,Ambur Star Briyani Since 1890,https://www.zomato.com/bangalore/ambur-star-br...,"Biryani, Chettinad, Chinese, North Indian",₹250 for one,4.2
9116,Kritunga Restaurant,https://www.zomato.com/bangalore/kritunga-rest...,"Andhra, South Indian, North Indian, Biryani, C...",₹150 for one,4.0


In [171]:
dff[dff['restaurant'] == 'Delhi Food Point' ]

,restaurant,link,cusine,price,rating
227,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹100 for one,3.6
1790,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹100 for one,3.6
2733,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹200 for one,3.6
4585,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹200 for one,3.6
6611,Delhi Food Point,https://www.zomato.com/bangalore/delhi-food-po...,"North Indian, Beverages",₹150 for one,3.6


In [172]:
dff1 = dff.drop_duplicates()

In [177]:
dff1 = dff1.reset_index(drop=True)

In [184]:
dff1['rest_id'] = np.arange(1,len(dff1)+1)

<IPython.core.display.Javascript object>

In [187]:
restaurant_info = dff1[['rest_id','restaurant','link','cusine','rating','price']]

In [192]:
restaurant_info

,rest_id,restaurant,link,cusine,rating,price
0,1,Ovenstory Pizza,https://www.zomato.com/bangalore/ovenstory-piz...,"Pizza, Fast Food",4.3,₹300 for one
1,2,Empire Restaurant,https://www.zomato.com/bangalore/empire-restau...,"North Indian, Biryani, Kebab, Chinese, South I...",4.0,₹300 for one
2,3,Leon's Burgers & Wings,https://www.zomato.com/bangalore/leons-burgers...,"Burger, Fast Food, Wraps, Street Food, Dessert...",4.3,₹300 for one
3,4,Truffles,https://www.zomato.com/bangalore/truffles-2-st...,"Burger, American, Bakery, Italian, Oriental, C...",4.4,₹300 for one
4,5,Domino's Pizza,https://www.zomato.com/bangalore/dominos-pizza...,"Pizza, Fast Food, Desserts, Beverages",4.1,₹300 for one
...,...,...,...,...,...,...
7809,7810,Arabian Hot Spot,https://www.zomato.com/bangalore/arabian-hot-s...,"Arabian, Mughlai, North Indian, Beverages, Juices",4.0,₹150 for one
7810,7811,Al-Bek,https://www.zomato.com/bangalore/al-bek-banasw...,"Mughlai, Arabian, North Indian, Chinese, Seafood",3.4,₹150 for one
7811,7812,Five Star Chicken,https://www.zomato.com/bangalore/five-star-chi...,"Rolls, Fast Food",4.1,₹150 for one
7812,7813,Kanti Sweets,https://www.zomato.com/bangalore/kanti-sweets-...,"Mithai, Street Food",4.2,₹150 for one


In [189]:
restaurant_info.to_csv('restaurant_info.csv',index=False)

In [193]:
rest_info = restaurant_info[restaurant_info['link'].duplicated() == False]

In [195]:
rest_info['rest_id'] = np.arange(1,len(rest_info)+1)

<IPython.core.display.Javascript object>

/tmp/ipykernel_6550/111581730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_info['rest_id'] = np.arange(1,len(rest_info)+1)


In [197]:
restaurant_info_new = rest_info.reset_index(drop=True)

In [200]:
restaurant_info_new.to_csv('restaurant_info.csv',index=False)

In [ ]:
['indiranagar','marathahlli','whitefield','kormangala','hsr','jayanagar','jp nagar','sarjapur road','lavelle road','malleshwaram','btm','mg road','kalyan nagar','electronic city','bannerghatta road','bellandur','new bel road','residency road','church street','brigade road','old airport road','banashankari','basavanagudi','rajajinagar','brookefield','kammanahalli']

In [56]:
len(urls)

5885

In [57]:
urls

['/bangalore/california-burrito-indiranagar/order',
 '/bangalore/dominos-pizza-indiranagar/order',
 '/bangalore/call-me-chow-kalyan-nagar-bangalore/order',
 '/bangalore/aubree-indiranagar-bangalore/order',
 '/bangalore/empire-restaurant-indiranagar/order',
 '/bangalore/tharavadu-koramangala-5th-block-bangalore/order',
 '/bangalore/kfc-indiranagar/order',
 '/bangalore/cheesecake-co-indiranagar-bangalore/order',
 '/bangalore/sri-udupi-park-indiranagar/order',
 '/bangalore/mcdonalds-indiranagar/order',
 '/bangalore/burger-king-indiranagar/order',
 '/bangalore/meghana-foods-indiranagar/order',
 '/bangalore/a2b-adyar-ananda-bhavan-indiranagar-bangalore/order',
 '/bangalore/ambur-star-briyani-since-1890-indiranagar-bangalore/order',
 '/bangalore/starbucks-coffee-indiranagar-bangalore/order',
 '/bangalore/pizza-hut-indiranagar/order',
 '/bangalore/kanti-sweets-indiranagar/order',
 '/bangalore/leons-burgers-wings-indiranagar-bangalore/order',
 '/bangalore/five-star-chicken-domlur-bangalore/ord

In [59]:
df = pd.DataFrame(urls)

In [61]:
df.to_csv('links.csv',index=None,header=['Links'])

In [ ]:
'indiranagar','marathahlli','whitefield','kormangala','hsr','jayanagar','jp nagar','sarjapur road','lavelle road','malleshwaram','btm','mg road','kalyan nagar','electronic city','bannerghatta road','bellandur','new bel road','residency road','church street','brigade road','old airport road','banashankari','basavanagudi','rajajinagar','brookefield',

In [75]:
for i in divs:
    name = i.select_one('h4')
    if name == None:
        continue
    else:
        print(name.text)

Leon's Burgers & Wings
Domino's Pizza
Call Me Chow
Chowman
McDonald's
Kwality Wall’s Frozen Dessert and Ice Cream Shop
Al Ameen Rolls Corner
N Razakhiya Hotel
Empire Restaurant


In [126]:
#ratings

for ss in divs:
    if ss.div.a == None:
        continue
    else:
        if ss.div.a.next_sibling == None:
            continue
        else:
            print(ss.div.a.next_sibling.div.div.div.div.div.div.div.text)

4.3
3.7
4.2
3.9
4.1
4.5
3.8
3.8
3.9


In [134]:
#Cuisinesd
for ss in divs:
    if ss.div.a == None:
        continue
    else:
        if ss.div.a.next_sibling == None:
            continue
        else:
            print(ss.div.a.next_sibling.div.next_sibling.p.text)

Burger, Fast Food, Wraps, Street Food, Desserts, Beverages
Pizza, Fast Food, Desserts, Beverages
Chinese, Japanese, Thai, Indonesian, Asian
Chinese, Asian, Thai, Oriental, Seafood, Sichuan, Beverages, Desserts
Burger, Fast Food, Beverages
Ice Cream, Desserts
Rolls, Fast Food
Rolls, Kebab
North Indian, Biryani, Kebab, Chinese, South Indian, Mughlai, Beverages


In [137]:
#price
for ss in divs:
    if ss.div.a == None:
        continue
    else:
        if ss.div.a.next_sibling == None:
            continue
        else:
            print(ss.div.a.next_sibling.div.next_sibling.p.next_sibling.text)


₹200 for one
₹200 for one
₹200 for one
₹200 for one
₹200 for one
₹200 for one
₹200 for one
₹200 for one
₹200 for one


In [190]:
import sys
results = [m.__name__ for m in sys.modules.values() if m]
results = sorted(results)
print(results)

['IPython', 'IPython.core', 'IPython.core.alias', 'IPython.core.application', 'IPython.core.async_helpers', 'IPython.core.autocall', 'IPython.core.builtin_trap', 'IPython.core.compilerop', 'IPython.core.completer', 'IPython.core.completerlib', 'IPython.core.crashhandler', 'IPython.core.debugger', 'IPython.core.display', 'IPython.core.display_functions', 'IPython.core.display_trap', 'IPython.core.displayhook', 'IPython.core.displaypub', 'IPython.core.error', 'IPython.core.events', 'IPython.core.excolors', 'IPython.core.extensions', 'IPython.core.formatters', 'IPython.core.getipython', 'IPython.core.history', 'IPython.core.hooks', 'IPython.core.inputtransformer2', 'IPython.core.interactiveshell', 'IPython.core.latex_symbols', 'IPython.core.logger', 'IPython.core.macro', 'IPython.core.magic', 'IPython.core.magic_arguments', 'IPython.core.magics', 'IPython.core.magics.auto', 'IPython.core.magics.basic', 'IPython.core.magics.code', 'IPython.core.magics.config', 'IPython.core.magics.display'